In [1]:
# importing required libraries
import os
import joblib
import numpy as np
import pandas as pd
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.preprocessing import scale
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from pprint import pprint

In [ ]:
# define
variable = '' # define varibale for taraining model
normalize_data = True # define true to normalize the data or false to used original data
scale_ = True # define true for scaling the data or false for not scaling the data

In [ ]:
# define path
base_path = ''
path = os.path.join(base_path, 'result')
file_path = os.path.join(path, 'final_df.csv')
plot = os.path.join(path, 'plot')

In [ ]:
# reading the dataset
data = pd.read_csv(file_path)
data.rename(columns = {'Unnamed: 0':'datacamp'}, inplace = True) # change the specific column name
# print(data)

In [ ]:
# normalize data
if normalize_data == True:
    data_final = data.drop(columns=['datacamp'],axis=1)
    # Square Transform
    square_transform = FunctionTransformer(lambda x: x ** 2)
    ct = ColumnTransformer(transformers=[['square_transform',square_transform,list(range(len(data_final.columns)))]],remainder='passthrough')
    square_X = ct.fit_transform(data_final).copy()
    square_X = pd.DataFrame(square_X,columns=list(data_final.columns)).copy()

    square_X.insert (0, 'datacamp', data['datacamp'])
    data = square_X

    # seperate the independent and target variable on training data
    x = square_X.drop(columns=[variable, 'datacamp'],axis=1)
    y = square_X[variable]
else:
    # seperate the independent and target variable on training data
    x = data.drop(columns=[variable, 'datacamp'],axis=1)
    y = data[variable]

In [ ]:
# scaling data
if scale_ == True:
    x_scale = scale(x)
    y_scale = scale(y)
    x_final = pd.DataFrame(x_scale, columns=list(x.columns))
    y_final = pd.Series(y_scale)
    data = pd.DataFrame({'datacamp': data['datacamp']})
    data = data.join(x_final)
    data[variable] = y_final
else:
    x_final = x
    y_final = y

# Split dataset into training set and test set
X_train, X_test, Y_train, Y_test = train_test_split(x_final, y_final, test_size=0.3) # 70% training and 30% test

In [ ]:
# creating the SVM regression model
kernel_list = ['linear', 'rbf', 'poly', 'sigmoid']

r2 = []
corr = []
p_value = []
for kernel_ in kernel_list:
    # Final SVM model for DS
    clf = make_pipeline(StandardScaler(), SVR(kernel=kernel_))

    #Creating the model on Training Data
    SVM=clf.fit(X_train,Y_train)
    prediction=SVM.predict(X_test)

    # creating df with predicted value and true value
    TestingDataResults=pd.DataFrame(data=X_test, columns=wavelengths)
    TestingDataResults['TargetColumn']=Y_test
    TestingDataResults['Prediction']=prediction
    model_r2 = r2_score(Y_test, prediction)
    r2.append(model_r2)
    
    corr_, value = pearsonr(prediction, Y_test)
    corr.append(corr_[0])
    p_value.append(value)
    
df_final = pd.DataFrame()
df_final['kernel'] = kernel_list
df_final['model_r2'] = r2
df_final['corr'] = corr
df_final['p_value'] = p_value
df_final

In [ ]:
# Final SVM model for DS
clf = make_pipeline(StandardScaler(), SVR(kernel='linear'))

#Creating the model on Training Data
SVM=clf.fit(X_train,Y_train)
prediction=SVM.predict(X_test)

#Printing some sample values of prediction
TestingDataResults=pd.DataFrame(data=X_test, columns=wavelengths)
TestingDataResults['TargetColumn']=Y_test
TestingDataResults['Prediction']=prediction
TestingDataResults.head()

In [ ]:
# Finding top 10 important wavelength
perm_importance = permutation_importance(SVM, X_test, Y_test)

features = np.array(wavelengths)

sorted_idx = perm_importance.importances_mean.argsort()
sorted_idx_ = sorted_idx[-10:]
plt.barh(features[sorted_idx_], perm_importance.importances_mean[sorted_idx_])
plt.xlabel("Permutation Importance")

In [ ]:
# export model 
model_filename = 'final_SVM.sav'
path = ""
model_ path = os.path.join(path, model_filename)
joblib.dump(RF_model, model_path)